# Backward Interval Partial Least Square (biPLS) Feature Selection (FS) Framework

In [ ]:
import os
import pandas as pd
import numpy as np

from scipy.signal import savgol_filter

from itertools import combinations
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

import matplotlib.pyplot as plt
from cycler import cycler
import matplotlib.transforms
import time
from joblib import dump
from joblib import load

# Classification
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

# Regression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
# Change to data directory

work_path = os.getcwd()
print(work_path)

os.chdir('YOUR DATA DIRECTORY')
data_path = os.getcwd()
print(data_path)
os.chdir(data_path)

In [ ]:
# Load data

os.chdir(data_path)

# Load dataset - df
df = pd.read_csv('YOUR DATA FILE.csv')

# Average spectra for each tissue type
avg_df_by_y = df.groupby(['target_y']).mean()
stdev_df_by_y = df.groupby(['target_y']).std()

# Column names to numerical
col_wavelengths = df.columns.drop('target_y')
col_wavelengths = col_wavelengths.astype(np.float64)
print('Number of wavelengths: ', len(col_wavelengths))

# Legend labels
tissue_types = df['target_y'].unique()
tissue_types.sort()

In [ ]:
# SG Smoothing of the raw data - Gentle smoothing with w/p = 2.5 to get rid of some baseline noise

# Features
X_raw = df.drop(['target_y'], axis=1)

# Target
y = df['target_y']

# Spectral Smoothing - Savitzky–Golay (SG) method
w = 5
p = 2
X_smooth = savgol_filter(X_raw, w, polyorder=p, axis=1, deriv=0)

# Smoothed dataframe
df_smooth = pd.DataFrame(X_smooth, columns = col_wavelengths.astype("string"))
df_smooth = pd.concat([df_smooth, y.rename('target_y')], axis=1)

# Average spectra for each tissue type
avg_df_by_y_smooth = df_smooth.groupby(['target_y']).mean()
stdev_df_by_y_smooth = df_smooth.groupby(['target_y']).std()

In [ ]:
# Change to save path
os.chdir(work_path)
os.chdir('YOUR SAVE PATH')
save_path = os.getcwd()
print(save_path)

#### Define functions

In [ ]:
# Define a function to calculate Variable Importance in Projection (VIP) in PLS
def vip(model):
    t = model.x_scores_
    w = model.x_weights_
    q = model.y_loadings_
    p, h = w.shape
    vips = np.zeros((p,))
    s = np.diag(t.T @ t @ q.T @ q).reshape(h, -1)
    total_s = np.sum(s)
    for i in range(p):
        weight = np.array([(w[i, j] / np.linalg.norm(w[:, j])) ** 2 for j in range(h)])
        vips[i] = np.sqrt(p * (s.T @ weight) / total_s)
    return vips

In [ ]:
# Define PLS regression function with cross validation
def plsr_cv(X, y, n_comp, cv):

    # Define cross validation
    cv = cv

    # Define scores as goodness-of-fit measures
    pls_y_preds = np.array([])
    pls_y_trues = np.array([])
    pls_r2_scores = []
    pls_mse_scores = []
    pls_rmse_scores = []

    # Evaluate model and cross validation
    count = 0
    for train_index, test_index in cv.split(X, y):

        count+=1
        # if count == cv.get_n_splits():
        #     print('CV finished')
        # Define PLS model
        pls_model = PLSRegression(n_components=n_comp)

        # Define train and test sets
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit PLS model to data
        pls_model.fit(X_train, y_train)
        # Predict using the fitted model
        y_pred = pls_model.predict(X_test)
        pls_y_preds = np.concatenate([pls_y_preds, y_pred], axis=None)
        # True y labels
        pls_y_trues = np.concatenate([pls_y_trues, y_test], axis=None)

        # Calculate regression scores
        pls_r2 = r2_score(y_test, y_pred)
        pls_mse = mean_squared_error(y_test, y_pred, squared=True)
        pls_rmse = mean_squared_error(y_test, y_pred, squared=False)

        # Store scores
        pls_r2_scores.append(pls_r2)
        pls_mse_scores.append(pls_mse)
        pls_rmse_scores.append(pls_rmse)

    return pls_y_preds, pls_y_trues, pls_r2_scores, pls_mse_scores, pls_rmse_scores

In [ ]:
def plsr_ncomp(X, y, n_comp, cv): # Change this to use RandomizedSearchCV

    # define the dictionary with parameters to test
    grid_param = {'n_components': (np.arange(n_comp) + 1).tolist()}

    # define PLS regression
    pls = PLSRegression()
    # define RandomizedSearchCV
    gs_pls = RandomizedSearchCV(pls, param_distributions=grid_param, scoring='neg_root_mean_squared_error',
                                cv=cv, return_train_score=True, n_jobs=-1)
    # fit GridSearchCV
    gs_pls.fit(X, y)

    # Best parameters and scores
    best_pls_ncomp = gs_pls.best_params_
    rmsecv_min = abs(gs_pls.best_score_)
    ind_pls_ncom = gs_pls.best_index_
    gs_cv_results = gs_pls.cv_results_

    return best_pls_ncomp, rmsecv_min, ind_pls_ncom, gs_cv_results

In [ ]:
# Define a function to set intervals by indexing
def interval_indexing(ind_list, interval_len):
    return [ind_list[i:i+interval_len] for i in range(0, len(ind_list), interval_len)]

In [ ]:
# Define a function to plot wavelength votes after optimization
def plot_wv_votes(wv, votes, tissue_1, tissue_2, save_flag):

    # Set the figure size
    f, ax = plt.subplots(figsize=(15, 8))

    # Tight layout
    f.tight_layout()

    # Plot
    plt.plot(wv, votes, '--o', linewidth=2, label='Wavelength Votes')

    # Set figure object
    ax.set_title('Wavelength Votes ' + str(tissue_1) + ' vs. ' + str(tissue_2))
    ax.set_xlabel('Wavelength (nm)')
    ax.set_ylabel('Votes (A.U.)')
    ax.set_xticks(np.arange(350, 1950, 100))
    ax.set_xlim([350, 1850])
    bbox = matplotlib.transforms.Bbox([[-0.2, -0.2], [15, 8.7]])

    # Save figure - condition
    if save_flag:
        if not os.path.exists(str(tissue_1) + '_' + str(tissue_2) + '_wv_votes.png'):
            f.savefig(os.getcwd() + '/' +  str(tissue_1) + '_' + str(tissue_2) + '_wv_votes.png', dpi = 1080, bbox_inches =bbox)
        plt.close()
    else:
        plt.show()

In [ ]:
# Define a function to evaluate clf accuracy using lda - Number of Features is a variable
# F1 score calculation can be added here

def clf_accuracy_eval(X, y, model, cv):

    # Initiate empty lists
    clf_accuracy_scores = [] # Calculate classification scores
    clf_balanced_scores = [] # Calculate balanced classification scores
    #clf_f1_scores = [] # Calculate F1 scores
    #count = 0
    for train_index, test_index in cv.split(X, y):
        # count+=1
        # print(count, " of ",  cv.get_n_splits(), " CV folds ", end="\r")

        # define model
        clf_model = model # model needs to be refined in each loop? so move down into the loop?

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit model
        clf_model.fit(X_train, y_train)
        # Predict using the fitted model
        y_predict = clf_model.predict(X_test)
        # Calculate classification accuracy score
        clf_score = accuracy_score(y_test, y_predict)
        # Calculate classification balanced accuracy score
        clf_balanced_score = balanced_accuracy_score(y_test, y_predict)
        # Calculate classification F-1 score
        #clf_f1_score = f1_score(y_test, y_predict, average='binary')
        # Store each iteration
        clf_accuracy_scores.append(clf_score)
        clf_balanced_scores.append(clf_balanced_score)
        #clf_f1_scores.append(clf_f1_score)

    return clf_accuracy_scores, clf_balanced_scores #, clf_f1_scores

#### The OVO Approach

In [ ]:
# Change to save path
os.chdir(save_path)
os.chdir('YOUR SAVE PATH - SUBFOLDER')
print(os.getcwd())

In [ ]:
# biPLS Framework for the OVO approach starts here:

# Define parameters
df_dataset = df_smooth
tissue_types = tissue_types

random_state = 42
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=random_state)

# Define binary combinations
list_tissue_comb = list(combinations(tissue_types,2))
# Save the list of binary pairs
if not os.path.exists('order_of_tissue_comb.joblib'):
    dump(list_tissue_comb, 'order_of_tissue_comb.joblib')

In [ ]:
# Run the for loop to calculate for all binary pairs

# for i,k in combinations(tissue_types,2): # can separate this to make an individual function
#
#     # Track timestamp - elapsed time
#     start_time = time.time()
#
#     count += 1

# if count == 2:
#     print('count = 2; break')
#     break

# Else for single pairs --->

i = 'boneCement'
k = 'cortBone'

# i = 'boneMarrow'
# k = 'cortBone'

# Select from the original dataset
binary_subset = df_dataset[df_dataset['target_y'].isin([i, k])]

# Features
X_binary_init = binary_subset.drop(['target_y'], axis=1)
# Target
y_binary_init = binary_subset['target_y']
# Convert to 0 and 1
y_binary_init = (y_binary_init == i).astype('uint8')


# Split Train Test Validation set
X_binary, X_val_all, y_binary, y_val_all = train_test_split(X_binary_init, y_binary_init,
                                                            test_size=0.2, stratify=y_binary_init, random_state=random_state)

# Save train test sets
dump(X_binary, str(i) + '_' + str(k) + '_X_binary.joblib')
dump(X_val_all, str(i) + '_' + str(k) + '_X_val_all.joblib')
dump(y_binary, str(i) + '_' + str(k) + '_y_binary.joblib')
dump(y_val_all, str(i) + '_' + str(k) + '_y_val_all.joblib')

In [ ]:
print('First label: ', i)
print('Second label: ',k)

# Index of all wavelengths
wavelength_ind = np.arange(0, len(col_wavelengths),1)

# Define the number of PLS components, random seed for cross validation, and wavelength interval length for optimization
pls_ncomp_opt = 30 # total number of 30 pls components to maximize
random_state_list = [42, 100124, 26224, 325749, 8260] # from random number generation
len_interval = [95, 80, 70, 60, 40, 20] # 16-25 intervals from lit; here is to somewhat match with the FWHM of LEDs

# Initiate a numpy array to store votes
wv_votes = np.zeros(len(col_wavelengths))

# Separator
sep = '_'

# Total time of execution
total_time = []

# Initiate dictionaries
pls_rmsecv_baseline = {}
all_avg_rmsecv = {}
all_std_rmsecv = {}
all_min_rmsecv = {}
all_min_rmsecv_ind = {}

# Initiate a dictionary of arrays to count how many times each wavelength has been retained for each binary combination
all_wv_votes = {}

# Track timestamp - elapsed time
start_time = time.time()

# Loop over different random seeds to get different compositions of cross validation
for random_cv in random_state_list:

    print('--- Random state = ', random_cv, ' ---')

    # Initiate nested dictionaries
    all_avg_rmsecv['Random State ' + str(random_cv)] = {}
    all_std_rmsecv['Random State ' + str(random_cv)] = {}
    all_min_rmsecv['Random State ' + str(random_cv)] = {}
    all_min_rmsecv_ind['Random State ' + str(random_cv)] = {}

    # Cross validation
    cv_baseline = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=random_cv)
    cv_opt = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=random_cv)

    # Calculate the optimal number of PLS components for baseline
    pls_ncomp_baseline, _, _, _ = plsr_ncomp(X_binary, y_binary, n_comp=pls_ncomp_opt,
                                             cv=10)  # GridSearchCV, cv = 10 always
    pls_best_ncomp_baseline = pls_ncomp_baseline['n_components']
    print('PLS best n_components baseline: ', pls_best_ncomp_baseline)

    # Calculate the PLS rmsecv for cv_baseline
    _, _, _, _, pls_rmse_scores = plsr_cv(X_binary, y_binary, n_comp=pls_best_ncomp_baseline, cv=cv_baseline)
    rmsecv_basline = np.mean(pls_rmse_scores)
    print('PLS Baseline RMSECV: %.5f +- %.5f' % (rmsecv_basline, np.std(pls_rmse_scores)))
    pls_rmsecv_baseline['Random State ' + str(random_cv)] = rmsecv_basline

    # Loop over different lengths to get different borders between intervals
    for length in len_interval:

        print('--- Interval length = ', length, ' ---')

        # Initiate nested dictionaries
        all_avg_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)] = {}
        all_std_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)] = {}
        all_min_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)] = {}
        all_min_rmsecv_ind['Random State ' + str(random_cv)]['Interval ' + str(length)] = {}

        # Set initial parameters
        rmsecv_init = np.copy(rmsecv_basline)
        X_binary_opt = X_binary.copy()
        ind_intervals_len = interval_indexing(wavelength_ind, length)  # Calculate index intervals based on the length
        wavelength_ind_opt = np.copy(wavelength_ind)

        print('Number of intervals = ', len(ind_intervals_len))

        # Initiate lists to store variables for the inner loop
        removed_ind_list_inner = []
        rest_ind_list_inner = []
        avg_rmse_scores = []
        avg_rmse_std = []

        for rep in range(len(ind_intervals_len)):

            print('--- Interval Rep = ', rep, ' ---')

            # Initiate nested dictionaries
            all_avg_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = {}
            all_std_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = {}
            all_min_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = {}
            all_min_rmsecv_ind['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = {}

            # Initiate lists to store variables for the inner loop
            removed_ind_list_inner = []
            rest_ind_list_inner = []
            avg_rmse_scores = []
            avg_rmse_std = []

            for wv_n in range(len(ind_intervals_len)):
                #print('Wavelength Interval Iteration #: ', wv_n)

                # Initiate parameters for the inner loop
                ind_intervals_inner = interval_indexing(wavelength_ind_opt, length)
                removed_ind_inner = ind_intervals_inner.pop(wv_n)
                removed_ind_list_inner.append(removed_ind_inner)
                #print('The removed index: ', removed_ind_inner)
                rest_ind_inner = np.concatenate(ind_intervals_inner).ravel()
                rest_ind_list_inner.append(rest_ind_inner)
                #print('The rest indices: ', rest_ind_inner)

                # Calculate the best number of PLS components
                pls_ncomp_inner, _, _, _ = plsr_ncomp(X_binary_opt.iloc[:, rest_ind_inner], y_binary,
                                                      n_comp=pls_ncomp_opt, cv=10)  # GridSearchCV, cv = 10 always
                pls_best_ncomp_inner = pls_ncomp_inner['n_components']
                #print('PLS best n_components inner: ', pls_best_ncomp_inner)

                _, _, _, _, pls_rmse_scores_inner = plsr_cv(X_binary_opt.iloc[:, rest_ind_inner], y_binary,
                                                            n_comp=pls_best_ncomp_inner, cv=cv_opt)
                avg_rmse_inner = np.mean(pls_rmse_scores_inner)
                #print('- biPLS RMSECV inner: %.5f +- %.5f' % (avg_rmse_inner, np.std(pls_rmse_scores_inner)))
                print('Iteration: %.0f; biPLS RMSECV inner: %.5f +- %.5f' % (wv_n, avg_rmse_inner, np.std(pls_rmse_scores_inner)))
                avg_rmse_scores.append(avg_rmse_inner)
                avg_rmse_std.append(np.std(pls_rmse_scores_inner))

            # Store all average PLS rmsecv and std for each random state and interval length
            all_avg_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = avg_rmse_scores
            all_std_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = avg_rmse_std

            # Calculate and store the minimum rmsecv values and its index
            min_rmsecv = np.min(avg_rmse_scores)
            min_rmsecv_ind = np.argmin(avg_rmse_scores)
            all_min_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = min_rmsecv
            all_min_rmsecv_ind['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = min_rmsecv_ind
            #print('The nth interval with lowest RMSECV: ', min_rmsecv_ind)

            if min_rmsecv < rmsecv_init:
                #print('Min RMSECV: ', min_rmsecv)
                #print('Initial RMSECV: ', rmsecv_init)
                #print('-> Yes smaller rmsecv')
                rmsecv_init = min_rmsecv  # set new baseline rmsecv
                print('-> New baseline rmsecv updated: ', rmsecv_init)
                # print('First wv of the deleted interval: ',
                #       X_binary_opt.iloc[:, removed_ind_list_inner[min_rmsecv_ind]].columns.astype(np.float64)[0])
                # Delete the wavelength interval whose removal results in the lowest RMSECV
                X_binary_opt = X_binary_opt.iloc[:, rest_ind_list_inner[min_rmsecv_ind]]
                # Delete the wavelength interval index
                removed_ind_opt = ind_intervals_len.pop(min_rmsecv_ind)
                # The remaining indices - to get length
                wavelength_ind_opt = np.concatenate(ind_intervals_len).ravel()
                # Update wavelength_ind for optimization
                wavelength_ind_opt = np.linspace(0, len(wavelength_ind_opt) - 1, len(wavelength_ind_opt), dtype=int)
                # Find the remained wavelengths
                wv_opt = X_binary_opt.columns.astype(np.float64)
                print('The number of remained wavelength (inner): ', len(wv_opt))
                # Find the index of remained wavelength in the original wavelength array
                wv_opt_ind = [idx for idx, value in enumerate(col_wavelengths) if value in wv_opt]
                # Count votes
                wv_votes[wv_opt_ind] += 1
            else:
                print('NO ALL GREATER; BREAK')
                print('--- End of optimization at Rep = ', rep, ' ---')
                break

# Store the votes
all_wv_votes = wv_votes

# Plot wavelength votes
plot_wv_votes(col_wavelengths, wv_votes, i, k, save_flag=True)

# End timestamp
end_time = time.time()
print('Time to run for each pair: ', (end_time - start_time) / 60, ' minutes')
total_time.append(end_time - start_time)
print('\n')

# Save variables
dump(wv_votes, str(i) + '_' + str(k) + '_wv_votes.joblib')

dump(pls_rmsecv_baseline, str(i) + '_' + str(k) + '_pls_rmsecv_baseline.joblib')
dump(all_avg_rmsecv, str(i) + '_' + str(k) + '_all_avg_rmsecv.joblib')
dump(all_std_rmsecv, str(i) + '_' + str(k) + '_all_std_rmsecv.joblib')
dump(all_min_rmsecv, str(i) + '_' + str(k) + '_all_min_rmsecv.joblib')
dump(all_min_rmsecv_ind, str(i) + '_' + str(k) + '_all_min_rmsecv_ind.joblib')
dump(total_time, str(i) + '_' + str(k) + '_total_time.joblib')
dump(X_binary, str(i) + '_' + str(k) + '_X_binary.joblib')
dump(X_val_all, str(i) + '_' + str(k) + '_X_val_all.joblib')
dump(y_binary, str(i) + '_' + str(k) + '_y_binary.joblib')
dump(y_val_all, str(i) + '_' + str(k) + '_y_val_all.joblib')

#### The OVR Approach

In [ ]:
# Change to save_path
os.chdir(save_path)
os.chdir('YOUR SAVE PATH - SUBFOLDER')
print(os.getcwd())

In [ ]:
# Define dataset
df_dataset = df_smooth
tissue_types = tissue_types

# Set global random state
random_state = 42

# Define cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=random_state)

# Bone Cement = 1 vs. [Cortical Bone, Trabecular Bone, Cartilage, Bone Marrow] = 0
i = 'boneCement'
k_1 = 'cortBone'
k_2 = 'traBone'
k_3 = 'cartilage'
k_4 = 'boneMarrow'

# Cortical Bone = 1 vs. [Trabecular Bone, Cartilage, Bone Marrow, Muscle] = 0
# i = 'cortBone'
# k_1 = 'traBone'
# k_2 = 'muscle'
# k_3 = 'cartilage'
# k_4 = 'boneMarrow'

print('First Label: ', i)
print('Second Label: ' + k_1 + ', ' + k_2 + ', ' + k_3 + ', ' + k_4)

# Data selection for multiple tissue types
df_subset_ovr = df_dataset[df_dataset['target_y'].isin([i, k_1, k_2, k_3, k_4])]
binary_subset = df_dataset[df_dataset['target_y'].isin([i, k_1, k_2, k_3, k_4])]
binary_subset.loc[df['target_y'].isin([k_1, k_2, k_3, k_4]), 'target_y'] = 'rest'
binary_subset.loc[df['target_y'].isin([i]), 'target_y'] = i
k = 'rest'

# Features
X_binary_init = binary_subset.drop(['target_y'], axis=1)
# Target
y_binary_init = binary_subset['target_y']
# Convert to 0 and 1 - i = 1, k = 0
y_binary_init = (y_binary_init == i).astype('uint8')

# Split Train Test Validation set
X_binary, X_val_all, y_binary, y_val_all = train_test_split(X_binary_init, y_binary_init,
                                                            test_size=0.2, stratify=y_binary_init,
                                                            random_state=random_state)

# Save train test sets
dump(X_binary, str(i) + '_' + str(k) + '_X_binary.joblib')
dump(X_val_all, str(i) + '_' + str(k) + '_X_val_all.joblib')
dump(y_binary, str(i) + '_' + str(k) + '_y_binary.joblib')
dump(y_val_all, str(i) + '_' + str(k) + '_y_val_all.joblib')

In [ ]:
print('First Label: ', i)
print('Second Label: ' + k_1 + ', ' + k_2 + ', ' + k_3 + ', ' + k_4)

# Index of all wavelengths
wavelength_ind = np.arange(0, len(col_wavelengths),1)

# Define the number of PLS components, random seed for cross validation, and wavelength interval length for optimization
pls_ncomp_opt = 30 # total number of 30 pls components to maximize
random_state_list = [42, 100124, 26224, 325749, 8260] # from random number generation
len_interval = [95, 80, 70, 60, 40, 20] # 16-25 intervals from lit; here is to somewhat match with the FWHM of LEDs

# Initiate a numpy array to store votes
wv_votes = np.zeros(len(col_wavelengths))

# Separator
sep = '_'

# Total time of execution
total_time = []

# Initiate dictionaries
pls_rmsecv_baseline = {}
all_avg_rmsecv = {}
all_std_rmsecv = {}
all_min_rmsecv = {}
all_min_rmsecv_ind = {}

# Initiate a dictionary of arrays to count how many times each wavelength has been retained for each binary combination
all_wv_votes = {}

# Track timestamp - elapsed time
start_time = time.time()

# Loop over different random seeds to get different compositions of cross validation
for random_cv in random_state_list:

    print('--- Random state = ', random_cv, ' ---')

    # Initiate nested dictionaries
    all_avg_rmsecv['Random State ' + str(random_cv)] = {}
    all_std_rmsecv['Random State ' + str(random_cv)] = {}
    all_min_rmsecv['Random State ' + str(random_cv)] = {}
    all_min_rmsecv_ind['Random State ' + str(random_cv)] = {}

    # Cross validation
    cv_baseline = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=random_cv)
    cv_opt = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=random_cv)

    # Calculate the optimal number of PLS components for baseline
    pls_ncomp_baseline, _, _, _ = plsr_ncomp(X_binary, y_binary, n_comp=pls_ncomp_opt,
                                             cv=10)  # GridSearchCV, cv = 10 always
    pls_best_ncomp_baseline = pls_ncomp_baseline['n_components']
    print('PLS best n_components baseline: ', pls_best_ncomp_baseline)

    # Calculate the PLS rmsecv for cv_baseline
    _, _, _, _, pls_rmse_scores = plsr_cv(X_binary, y_binary, n_comp=pls_best_ncomp_baseline, cv=cv_baseline)
    rmsecv_basline = np.mean(pls_rmse_scores)
    print('PLS Baseline RMSECV: %.5f +- %.5f' % (rmsecv_basline, np.std(pls_rmse_scores)))
    pls_rmsecv_baseline['Random State ' + str(random_cv)] = rmsecv_basline

    # Loop over different lengths to get different borders between intervals
    for length in len_interval:

        print('--- Interval length = ', length, ' ---')

        # Initiate nested dictionaries
        all_avg_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)] = {}
        all_std_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)] = {}
        all_min_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)] = {}
        all_min_rmsecv_ind['Random State ' + str(random_cv)]['Interval ' + str(length)] = {}

        # Set initial parameters
        rmsecv_init = np.copy(rmsecv_basline)
        X_binary_opt = X_binary.copy()
        ind_intervals_len = interval_indexing(wavelength_ind, length)  # Calculate index intervals based on the length
        wavelength_ind_opt = np.copy(wavelength_ind)

        print('Number of intervals = ', len(ind_intervals_len))

        # Initiate lists to store variables for the inner loop
        removed_ind_list_inner = []
        rest_ind_list_inner = []
        avg_rmse_scores = []
        avg_rmse_std = []

        for rep in range(len(ind_intervals_len)):

            print('--- Interval Rep = ', rep, ' ---')

            # Initiate nested dictionaries
            all_avg_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = {}
            all_std_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = {}
            all_min_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = {}
            all_min_rmsecv_ind['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = {}

            # Initiate lists to store variables for the inner loop
            removed_ind_list_inner = []
            rest_ind_list_inner = []
            avg_rmse_scores = []
            avg_rmse_std = []

            for wv_n in range(len(ind_intervals_len)):
                #print('Wavelength Interval Iteration #: ', wv_n)

                # Initiate parameters for the inner loop
                ind_intervals_inner = interval_indexing(wavelength_ind_opt, length)
                removed_ind_inner = ind_intervals_inner.pop(wv_n)
                removed_ind_list_inner.append(removed_ind_inner)
                #print('The removed index: ', removed_ind_inner)
                rest_ind_inner = np.concatenate(ind_intervals_inner).ravel()
                rest_ind_list_inner.append(rest_ind_inner)
                #print('The rest indices: ', rest_ind_inner)

                # Calculate the best number of PLS components
                pls_ncomp_inner, _, _, _ = plsr_ncomp(X_binary_opt.iloc[:, rest_ind_inner], y_binary,
                                                      n_comp=pls_ncomp_opt, cv=10)  # GridSearchCV, cv = 10 always
                pls_best_ncomp_inner = pls_ncomp_inner['n_components']
                #print('PLS best n_components inner: ', pls_best_ncomp_inner)

                _, _, _, _, pls_rmse_scores_inner = plsr_cv(X_binary_opt.iloc[:, rest_ind_inner], y_binary,
                                                            n_comp=pls_best_ncomp_inner, cv=cv_opt)
                avg_rmse_inner = np.mean(pls_rmse_scores_inner)
                #print('- biPLS RMSECV inner: %.5f +- %.5f' % (avg_rmse_inner, np.std(pls_rmse_scores_inner)))
                print('Iteration: %.0f; biPLS RMSECV inner: %.5f +- %.5f' % (wv_n, avg_rmse_inner, np.std(pls_rmse_scores_inner)))
                avg_rmse_scores.append(avg_rmse_inner)
                avg_rmse_std.append(np.std(pls_rmse_scores_inner))

            # Store all average PLS rmsecv and std for each random state and interval length
            all_avg_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = avg_rmse_scores
            all_std_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = avg_rmse_std

            # Calculate and store the minimum rmsecv values and its index
            min_rmsecv = np.min(avg_rmse_scores)
            min_rmsecv_ind = np.argmin(avg_rmse_scores)
            all_min_rmsecv['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = min_rmsecv
            all_min_rmsecv_ind['Random State ' + str(random_cv)]['Interval ' + str(length)]['Rep ' + str(rep)] = min_rmsecv_ind
            #print('The nth interval with lowest RMSECV: ', min_rmsecv_ind)

            if min_rmsecv < rmsecv_init:
                #print('Min RMSECV: ', min_rmsecv)
                #print('Initial RMSECV: ', rmsecv_init)
                #print('-> Yes smaller rmsecv')
                rmsecv_init = min_rmsecv  # set new baseline rmsecv
                print('-> New baseline rmsecv updated: ', rmsecv_init)
                # print('First wv of the deleted interval: ',
                #       X_binary_opt.iloc[:, removed_ind_list_inner[min_rmsecv_ind]].columns.astype(np.float64)[0])
                # Delete the wavelength interval whose removal results in the lowest RMSECV
                X_binary_opt = X_binary_opt.iloc[:, rest_ind_list_inner[min_rmsecv_ind]]
                # Delete the wavelength interval index
                removed_ind_opt = ind_intervals_len.pop(min_rmsecv_ind)
                # The remaining indices - to get length
                wavelength_ind_opt = np.concatenate(ind_intervals_len).ravel()
                # Update wavelength_ind for optimization
                wavelength_ind_opt = np.linspace(0, len(wavelength_ind_opt) - 1, len(wavelength_ind_opt), dtype=int)
                # Find the remained wavelengths
                wv_opt = X_binary_opt.columns.astype(np.float64)
                print('The number of remained wavelength (inner): ', len(wv_opt))
                # Find the index of remained wavelength in the original wavelength array
                wv_opt_ind = [idx for idx, value in enumerate(col_wavelengths) if value in wv_opt]
                # Count votes
                wv_votes[wv_opt_ind] += 1
            else:
                print('NO ALL GREATER; BREAK')
                print('--- End of optimization at Rep = ', rep, ' ---')
                break

# Store the votes
all_wv_votes = wv_votes

# Plot wavelength votes
plot_wv_votes(col_wavelengths, wv_votes, i, k, save_flag=True)

# End timestamp
end_time = time.time()
print('Time to run for each pair: ', (end_time - start_time) / 60, ' minutes')
total_time.append(end_time - start_time)
print('\n')

# Save variables
dump(wv_votes, str(i) + '_' + str(k) + '_wv_votes.joblib')
# Save variables
#dump(all_wv_votes, 'all_wv_votes.joblib')
dump(pls_rmsecv_baseline, str(i) + '_' + str(k) + '_pls_rmsecv_baseline.joblib')
dump(all_avg_rmsecv, str(i) + '_' + str(k) + '_all_avg_rmsecv.joblib')
dump(all_std_rmsecv, str(i) + '_' + str(k) + '_all_std_rmsecv.joblib')
dump(all_min_rmsecv, str(i) + '_' + str(k) + '_all_min_rmsecv.joblib')
dump(all_min_rmsecv_ind, str(i) + '_' + str(k) + '_all_min_rmsecv_ind.joblib')
dump(total_time, str(i) + '_' + str(k) + '_total_time.joblib')

#### Find the highest vote and the final feature subset
- This is for the OVR approach
- Same applies to the OVO approach

In [ ]:
# Change to save_path
os.chdir(save_path)
os.chdir('YOUR SAVE PATH - SUBFOLDER')
print(os.getcwd())

In [ ]:
# One vs Rest

# Votes
boneCement_rest_ovr_votes = load('YOUR VOTES.joblib')
cortBone_rest_ovr_votes = load('YOUR VOTES.joblib')

# X Train and Validation Dataset
boneCement_X_train = load('YOUR TRAIN/TEST X SET.joblib')
boneCement_y_train = load('YOUR TRAIN/TEST y SET.joblib')
boneCement_X_val = load('YOUR VALIDATION X SET.joblib')
boneCement_y_val = load('YOUR VALIDATION y SET.joblib')

cortBone_X_train = load('YOUR TRAIN/TEST X SET.joblib')
cortBone_y_train = load('YOUR TRAIN/TEST y SET.joblib')
cortBone_X_val = load('YOUR VALIDATION X SET.joblib')
cortBone_y_val = load('YOUR VALIDATION y SET.joblib')

In [ ]:
# Define dataset
df_dataset = df_smooth
tissue_types = tissue_types

# Set global random state
random_state = 42

# Define cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=random_state)

# Bone Cement = 1 vs. [Cortical Bone, Trabecular Bone, Cartilage, Bone Marrow] = 0
i = 'boneCement'
k_1 = 'cortBone'
k_2 = 'traBone'
k_3 = 'cartilage'
k_4 = 'boneMarrow'

# Cortical Bone = 1 vs. [Trabecular Bone, Cartilage, Bone Marrow, Muscle] = 0
# i = 'cortBone'
# k_1 = 'traBone'
# k_2 = 'muscle'
# k_3 = 'cartilage'
# k_4 = 'boneMarrow'

k = 'rest'
print('First Label: ', i)
#print('Second Label: ',k)
print('Second Label: ' + k_1 + ', ' + k_2 + ', ' + k_3 + ', ' + k_4)

# Data selection for multiple tissue types
df_subset_ovr = df_dataset[df_dataset['target_y'].isin([i, k_1, k_2, k_3, k_4])]

# Average spectra for each tissue type
avg_df_select_tissues = df_subset_ovr.groupby(['target_y']).mean()
stdev_df_select_tissues = df_subset_ovr.groupby(['target_y']).std()


In [ ]:
# Set variables

# Bone Cement
votes_array = boneCement_rest_ovr_votes
X_train = boneCement_X_train
y_train = boneCement_y_train
X_val = boneCement_X_val
y_val = boneCement_y_val

# Cortical Bone
# votes_array = cortBone_rest_ovr_votes
# X_train = cortBone_X_train
# y_train = cortBone_y_train
# X_val = cortBone_X_val
# y_val = cortBone_y_val

In [ ]:
# Sort in descending order
votes_rank_idx = np.argsort(-votes_array)

# Construct a dataframe
votes_ranked_features = pd.DataFrame({'Features': col_wavelengths[votes_rank_idx],
                                      'Votes': votes_array[votes_rank_idx],
                                      'Ranking_idx': votes_rank_idx})

# Select the highest VOTES
select_top_votes = votes_ranked_features[votes_ranked_features["Votes"] == max(votes_array)]

# Selected X and y train
X_train_select = X_train.iloc[:, select_top_votes["Ranking_idx"].values]

# Find the optimal number of PLS components out of 30
pls_ncomp_opt = 30
pls_ncomp_init, _, _, _ = plsr_ncomp(X_train_select, y_train, n_comp=pls_ncomp_opt, cv=10)
pls_best_ncomp = pls_ncomp_init['n_components']
print('PLS Number of Componenets: ', pls_best_ncomp)

# Calculate PLS-VIP scores
pls_vip = PLSRegression(n_components=pls_best_ncomp).fit(X_train_select, y_train)
vip_coeffs = vip(pls_vip)

# Selected wavelengths
select_wv = X_train_select.columns.values.astype(np.float64)

select_top_votes.insert(len(select_top_votes.columns), 'PLS-VIP', vip_coeffs)

In [ ]:
# Set the figure size
f, ax = plt.subplots(figsize=(10, 8))

# Tight layout
f.tight_layout()

# Plot
plt.scatter(select_wv, vip_coeffs)
ax.set_xlim([350, 1850])
ax.set_title('PLS VIP Scores for Top Votes ' + str(i) + ' vs. ' + str(k))
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('PLS VIP Scores (A.U.)')
bbox = matplotlib.transforms.Bbox([[-0.45, -1.3], [11.45, 8.56]])

if not os.path.exists(str(i) + '_' + str(k) + '_pls_vip.png'):
    f.savefig(os.getcwd() + '/' +  str(i) + '_' + str(k) + '_pls_vip.png', dpi = 1080, bbox_inches =bbox)
plt.close()

In [ ]:
# Set the number of features for the final subset
K_features = 10

# Initiate the search
selected_ind_update = [select_top_votes['Ranking_idx'][np.argmax(vip_coeffs)]]
select_vip_ind_update = np.copy(select_top_votes['Ranking_idx'])
select_vip_ind_update = np.delete(select_vip_ind_update, np.argmax(vip_coeffs))

while len(selected_ind_update) < K_features:
    #print('Number of Selected Ind: ', len(selected_ind_update))
    pearson_rank = []
    for n in range(len(select_vip_ind_update)):
        pearson_corr = np.max(np.abs(np.corrcoef(X_train.iloc[:, selected_ind_update + [select_vip_ind_update[n]]], rowvar=False))
                              - np.identity(len(selected_ind_update) + 1))
        pearson_rank.append(pearson_corr)
    selected_ind = np.argsort(pearson_rank)[0]
    selected_select_vip_ind = select_vip_ind_update[selected_ind]
    #print('Selected Ind: ', selected_anova_ind)
    #print('Selected Pearson Coeff: ', pearson_rank[selected_ind])
    selected_ind_update.append(selected_select_vip_ind)
    select_vip_ind_update = np.delete(select_vip_ind_update, selected_ind)

# Find the final selected wavelengths
pls_vip_select_wv_final = col_wavelengths[selected_ind_update].values
pls_vip_select_wv_final_str = [str(x) for x in pls_vip_select_wv_final]

# Save
dump(pls_vip_select_wv_final, str(i) + '_' + str(k) + '_pls_vip_select_wv_final.joblib')

In [ ]:
# Plot

# Set the figure size
f, ax = plt.subplots(figsize=(10, 8))

# Tight layout
f.tight_layout()

# Selected Tissue Types
select_tissue_label = [i,k_1,k_2,k_3,k_4]
select_tissue_label.sort()

# Set custom color cycle
if i == 'boneCement':
    custom_cycler= (cycler('color', ['#d62728', '#808080', '#808080', '#808080', '#808080']) + cycler(lw=[1.5, 1.5, 1.5, 1.5, 1.5]))
elif i == 'cortBone':
    custom_cycler = (cycler('color', ['#808080', '#808080', '#d62728', '#808080', '#808080']) + cycler(lw=[1.5, 1.5, 1.5, 1.5, 1.5]))

# Set colors
ax.set_prop_cycle(custom_cycler)

for tissue in range(avg_df_select_tissues.shape[0]):
    plt.plot(col_wavelengths, avg_df_select_tissues.iloc[tissue, :], label=select_tissue_label[tissue])
    pos_std = avg_df_select_tissues.iloc[tissue, :] + stdev_df_select_tissues.iloc[tissue, :]
    neg_std = avg_df_select_tissues.iloc[tissue, :] - stdev_df_select_tissues.iloc[tissue, :]
    ax.fill_between(col_wavelengths, neg_std, pos_std, alpha=0.08)

for wv in pls_vip_select_wv_final:
    plt.axvline(x=wv, color='g', linestyle='-', alpha=0.3)

# Text
plt.text(0, -0.13, 'Selected Wavelengths - Final: ' + str(np.array(pls_vip_select_wv_final)), horizontalalignment='left',
         verticalalignment = 'center_baseline', transform=ax.transAxes)

# Set figure object
ax.set_xlim([350, 1850])
ax.set_title('Selected Wavelengths for ' + str(i) + ' vs. ' + str(k))
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel('Normalized Intensity (A.U.)')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

# Save
bbox = matplotlib.transforms.Bbox([[-0.45, -1.3], [11.45, 8.56]])
if not os.path.exists(str(i) + '_' + str(k) + 'pls_vip_select_wv_final_plot.png'):
    f.savefig(os.getcwd() + '/' +  str(i) + '_' + str(k) + 'pls_vip_select_wv_final_plot.png', dpi = 1080, bbox_inches =bbox)
plt.close()

#### Plot Accuracy vs. Number of Features in the order given by the algorithm
- or can rank the order by f_classif scores

In [ ]:
# Final Wavelengths

boneCement_wv = load('YOUR PLS VIP SELECTED FEATURES.joblib')
boneCement_wv = [str(x) for x in boneCement_wv]
boneCement_wv = np.array(boneCement_wv)
cortBone_wv = load('YOUR PLS VIP SELECTED FEATURES.joblib')
cortBone_wv = [str(x) for x in cortBone_wv]
cortBone_wv = np.array(cortBone_wv)

In [ ]:
# Bone Cement
i = 'boneCement'
k = 'rest'
final_wv = boneCement_wv
selected_X_train = X_train.loc[:, final_wv]
selected_X_val = X_val.loc[:, final_wv]

# Cortical Bone
# i = 'cortBone'
# k = 'rest'
# final_wv = cortBone_wv
# selected_X_train = X_train.loc[:, final_wv]
# selected_X_val = X_val.loc[:, final_wv]

In [ ]:
# Rank by f_classif

# Select the top feature by ANOVA F-value
fs = SelectKBest(f_classif,k="all")
fs.fit(selected_X_train,y_train)
# Find the index from high to low
f_idx = np.argsort(-fs.scores_)
f_rank_wv = final_wv[f_idx]
dump(f_rank_wv, str(i) + '_' + str(k) + '_pls_vip_wv_f_rank.joblib')

# f ranked wavelengths
selected_X_train = X_train.loc[:, f_rank_wv]
selected_X_val = X_val.loc[:, f_rank_wv]

In [ ]:
# Calculate clf accuracy

# Define cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=random_state)

all_lda_balanced_scores = []
all_lda_balanced_stdev = []
all_lda_balanced_scores_val = []
all_lda_balanced_stdev_val = []

for wv in range(len(final_wv)):
    # Define LDA model
    model = LinearDiscriminantAnalysis()

    # Calculate clf accuracy cv
    print('Number of Features: ', wv+1)
    _, lda_balanced_scores = clf_accuracy_eval(selected_X_train.iloc[:,0:wv+1], y_train, model, cv)
    _, lda_balanced_scores_val = clf_accuracy_eval(selected_X_val.iloc[:,0:wv+1], y_val, model, cv)

    all_lda_balanced_scores.append(np.mean(lda_balanced_scores))
    all_lda_balanced_stdev.append(np.std(lda_balanced_scores))

    all_lda_balanced_scores_val.append(np.mean(lda_balanced_scores_val))
    all_lda_balanced_stdev_val.append(np.std(lda_balanced_scores_val))

# Save Variables
dump(all_lda_balanced_scores, str(i) + '_' + str(k) + '_lda_balanced_train_f_rank.joblib')
dump(all_lda_balanced_stdev, str(i) + '_' + str(k) + '_lda_balanced_std_train_f_rank.joblib')

dump(all_lda_balanced_scores_val, str(i) + '_' + str(k) + '_lda_balanced_val_f_rank.joblib')
dump(all_lda_balanced_stdev_val, str(i) + '_' + str(k) + '_lda_balanced_std_val_f_rank.joblib')

In [ ]:
# plot Clf Accuracy vs. Number of Features

# Set the figure size
f, ax = plt.subplots(figsize=(8, 6))
# Tight layout
f.tight_layout()

# Plot - Balanced accuracy on train
plt.plot(np.linspace(1, len(f_rank_wv), len(f_rank_wv)), all_lda_balanced_scores, '--', lw=2.5)
plt.errorbar(np.linspace(1, len(f_rank_wv), len(f_rank_wv)), all_lda_balanced_scores,
             yerr=all_lda_balanced_stdev,
             fmt='o', capsize=1.5)
xlabels = np.append(np.array('0'), f_rank_wv)
plt.xticks(np.arange(0, len(f_rank_wv) + 1, 1), xlabels)
# Set figure object
ax.set_title('Classification Accuracy vs. Number of Wavelengths for ' + str(i) + ' vs. ' + str(k))
ax.set_xlabel('Wavelengths Included (nm)')
ax.set_ylabel('LDA Balanced Clf Accuracy')
#ax.set_xlim([350, 1850])
bbox = matplotlib.transforms.Bbox([[-0.4, -0.4], [8.5, 6.5]])

# Save figure
if not os.path.exists(str(i) + '_' + str(k) + '_balanced_nFeatures_train_f_rank.png'):
    f.savefig(os.getcwd() + '/' + str(i) + '_' + str(k) + '_balanced_nFeatures_train_f_rank.png', dpi=1080,
              bbox_inches=bbox)


# plot Clf Accuracy vs. Number of Features

# Set the figure size
f, ax = plt.subplots(figsize=(8, 6))
# Tight layout
f.tight_layout()

# Plot - Balanced accuracy on validation
plt.plot(np.linspace(1, len(f_rank_wv), len(f_rank_wv)), all_lda_balanced_scores_val, '--', lw=2.5)
plt.errorbar(np.linspace(1, len(f_rank_wv), len(f_rank_wv)), all_lda_balanced_scores_val,
             yerr=all_lda_balanced_stdev_val,
             fmt='o', capsize=1.5)
xlabels = np.append(np.array('0'), f_rank_wv)
plt.xticks(np.arange(0, len(f_rank_wv) + 1, 1), xlabels)
# Set figure object
ax.set_title('Classification Accuracy vs. Number of Wavelengths for ' + str(i) + ' vs. ' + str(k))
ax.set_xlabel('Wavelengths Included (nm)')
ax.set_ylabel('LDA Balanced Clf Accuracy')
#ax.set_xlim([350, 1850])
bbox = matplotlib.transforms.Bbox([[-0.4, -0.4], [8.5, 6.5]])

# Save figure
if not os.path.exists(str(i) + '_' + str(k) + '_balanced_nFeatures_val_f_rank.png'):
    f.savefig(os.getcwd() + '/' + str(i) + '_' + str(k) + '_balanced_nFeatures_val_f_rank.png', dpi=1080, bbox_inches=bbox)